In [30]:
import os
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re

# incruit 디렉토리 확인 및 생성
if not os.path.exists('./incruit'):
    os.makedirs('./incruit')  # 디렉토리 생성

def log_process(message):
    """프로세스 상태를 incruit/makelog_process_YYYYMMDD.log 파일에 기록하고 화면에 출력"""
    today = datetime.today().strftime('%Y%m%d')
    log_file_name = f'./incruit/makelog_process_{today}.log'
    print(message)
    with open(log_file_name, 'a', encoding='utf-8') as process_file:
        timestamp = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        process_file.write(f"{timestamp},{message}\n")

job_url_list = {
    "DE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno={}"
    ],
    "FE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%C7%C1%B7%D0%C6%AE&startno={}"
    ],
    "BE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B9%E9%BF%A3%B5%E5&startno={}"
    ],
    "DA": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BA%D0%BC%AE%B0%A1&startno={}"
    ],
    "MLE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B8%D3%BD%C5%B7%AF%B4%D7+%BF%A3%C1%F6%B4%CF%BE%EE&startno={}"
    ]
}

try:
    # 오늘 날짜로 로그 파일 이름 설정
    today = datetime.today().strftime('%Y%m%d')

    # 로그 파일을 찾을 디렉토리 설정
    log_directory = './incruit'
    log_files = [f for f in os.listdir(log_directory) if re.match(r'^\d{8}\.log$', f)]

    # 가장 최근에 생성된 로그 파일 찾기
    if log_files:
        log_files.sort(key=lambda x: os.path.getmtime(os.path.join(log_directory, x)), reverse=True)
        recent_log_file_name = log_files[0]  # 가장 최근의 로그 파일을 선택
        log_process(f"Found the most recent log file: {recent_log_file_name}")
    else:
        log_process("No log files found in the directory.")
        recent_log_file_name = None  # recent_log_file_name을 None으로 설정

    # 이전 로그 파일이 존재하는지 확인하고 읽기
    previous_urls = {}  # 이전 로그에 있는 URL
    if recent_log_file_name and os.path.exists(os.path.join(log_directory, recent_log_file_name)):
        with open(os.path.join(log_directory, recent_log_file_name), 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if lines:  # 파일에 내용이 있을 때만 처리
                log_process(f"Reading existing log file: {recent_log_file_name}")
                for line in lines[1:]:  # 첫 번째 줄은 header
                    columns = line.strip().split(',')
                    if len(columns) >= 5:  # 각 열이 다 있다면
                        url = columns[1]
                        previous_urls[url] = columns[0]  # 해당 URL의 직무를 저장
    else:
        log_process("No previous log file to read or file is empty.")

    # 오늘 크롤링한 URL을 수집
    all_links = []  # 오늘 수집한 모든 링크

    # 각 job (키값)에 대한 URL 처리
    for job_key, urls in job_url_list.items():
        for url in urls:
            start_page = 0
            end_page = 38
            page_step = 30  # 페이지네이션의 startno는 30씩 증가

            # 페이지 순차적으로 크롤링
            for page in range(start_page, end_page):
                startno = page * page_step
                formatted_url = url.format(startno)

                log_process(f"Fetching URL: {formatted_url} (page {page})")
                # HTTP 요청 보내기
                response = requests.get(formatted_url)
                soup = BeautifulSoup(response.text, 'html.parser')

                # 직무 공고 링크 추출 (jobdb_info 클래스에 포함된 a 태그)
                job_links_on_page = soup.find_all('a', href=True)

                for link_tag in job_links_on_page:
                    link = link_tag['href']
                    if "jobdb_info" in link:
                        full_link = "https://www.incruit.com" + link if link.startswith('/') else link
                        if full_link not in all_links:
                            all_links.append(full_link)

    log_process("Crawling completed. Now processing URLs.")

    # 오늘 수집된 URL을 'today_crawled_urls_YYYYMMDD.txt'에 저장
    today_crawled_file_name = f'./incruit/today_crawled_urls_{today}.txt'
    with open(today_crawled_file_name, 'w', encoding='utf-8') as file:
        for url in all_links:
            file.write(url + "\n")
    log_process(f"URLs written to {today_crawled_file_name}.")

    # 어제 로그에 존재했던 URL을 'yesterday_existing_urls_YYYYMMDD.txt'에 저장
    yesterday_existing_file_name = f'./incruit/yesterday_existing_urls_{today}.txt'
    with open(yesterday_existing_file_name, 'w', encoding='utf-8') as file:
        for url in previous_urls.keys():
            file.write(url + "\n")
    log_process(f"URLs written to {yesterday_existing_file_name}.")

except Exception as e:
    # 오류 발생 시 오류 메시지 기록
    error_message = str(e)
    log_process(f"An error occurred: {error_message}")  
    
    
# 불러온 url 목록보기


Found the most recent log file: 20241222.log
Reading existing log file: 20241222.log
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=0 (page 0)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=30 (page 1)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=60 (page 2)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=90 (page 3)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=120 (page 4)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=150 (page 5)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=18

KeyboardInterrupt: 

In [34]:
import os
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re

# incruit 디렉토리 확인 및 생성
if not os.path.exists('./incruit'):
    os.makedirs('./incruit')  # 디렉토리 생성

def log_error(error_message):
    """오류를 incruit/makelog_err_YYYYMMDD.log 파일에 기록"""
    today = datetime.today().strftime('%Y%m%d')
    log_file_name = f'./incruit/makelog_err_{today}.log'
    with open(log_file_name, 'a', encoding='utf-8') as err_file:
        timestamp = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        err_file.write(f"{timestamp},{error_message}\n")

def log_process(message):
    """프로세스 상태를 incruit/makelog_process_YYYYMMDD.log 파일에 기록하고 화면에 출력"""
    today = datetime.today().strftime('%Y%m%d')
    log_file_name = f'./incruit/makelog_process_{today}.log'
    print(message)
    with open(log_file_name, 'a', encoding='utf-8') as process_file:
        timestamp = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        process_file.write(f"{timestamp},{message}\n")

job_url_list = {
    "DE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno={}"
    ],
    "FE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%C7%C1%B7%D0%C6%AE&startno={}"
    ],
    "BE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B9%E9%BF%A3%B5%E5&startno={}"
    ],
    "DA": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BA%D0%BC%AE%B0%A1&startno={}"
    ],
    "MLE": [
        "https://search.incruit.com/list/search.asp?col=job&kw=%B8%D3%BD%C5%B7%AF%B4%D7+%BF%A3%C1%F6%B4%CF%BE%EE&startno={}"
    ]
}

try:
    # 오늘 날짜로 로그 파일 이름 설정
    today = datetime.today().strftime('%Y%m%d')
    today_log_file_name = f"./incruit/{today}.log"

    # 로그 파일을 찾을 디렉토리 설정
    log_directory = './incruit'
    log_files = [f for f in os.listdir(log_directory) if re.match(r'^\d{8}\.log$', f)]

    # 가장 최근에 생성된 로그 파일 찾기
    if log_files:
        log_files.sort(key=lambda x: os.path.getmtime(os.path.join(log_directory, x)), reverse=True)
        recent_log_file_name = log_files[0]  # 가장 최근의 로그 파일을 선택
        log_process(f"Found the most recent log file: {recent_log_file_name}")
    else:
        log_process("No log files found in the directory. All URLs will be marked as 'update'.")
        recent_log_file_name = None  # recent_log_file_name을 None으로 설정

    # 이전 로그 파일이 존재하는지 확인하고 읽기
    previous_urls = {}  # 이전 로그에 있는 URL 및 해당 job
    if recent_log_file_name and os.path.exists(os.path.join(log_directory, recent_log_file_name)):
        with open(os.path.join(log_directory, recent_log_file_name), 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if lines:
                log_process(f"Reading existing log file: {recent_log_file_name}")
                for line in lines[1:]:  # 첫 번째 줄은 header
                    columns = line.strip().split(',')
                    if len(columns) >= 5:
                        url = columns[1]
                        job = columns[0]  # 해당 URL의 job
                        notice_status = columns[2]
                        if notice_status != "deleted":  # "deleted" 상태인 URL은 제외
                            previous_urls[url] = {
                                'job': job,
                                'notice_status': notice_status,
                                'work_status': columns[3],
                                'done_time': columns[4]
                            }
    else:
        log_process("No previous log file to read or file is empty.")

    # 오늘 크롤링한 URL을 수집
    all_links = []  # 오늘 수집한 모든 링크
    job_for_links = {}  # 각 링크에 해당하는 job을 기록하기 위한 dictionary

    # 각 job (키값)에 대한 URL 처리
    for job_key, urls in job_url_list.items():
        for url in urls:
            start_page = 0
            end_page = 38
            page_step = 30  # 페이지네이션의 startno는 30씩 증가

            # 페이지 순차적으로 크롤링
            for page in range(start_page, end_page):
                startno = page * page_step
                formatted_url = url.format(startno)

                log_process(f"Fetching URL: {formatted_url} (page {page})")
                # HTTP 요청 보내기
                response = requests.get(formatted_url)
                soup = BeautifulSoup(response.text, 'html.parser')

                # 직무 공고 링크 추출 (jobdb_info 클래스에 포함된 a 태그)
                job_links_on_page = soup.find_all('a', href=True)

                for link_tag in job_links_on_page:
                    link = link_tag['href']
                    if "jobdb_info" in link:
                        full_link = "https://www.incruit.com" + link if link.startswith('/') else link
                        if full_link not in all_links:
                            all_links.append(full_link)
                            job_for_links[full_link] = job_key  # 링크에 해당하는 job 기록

    log_process("Crawling completed. Now processing URLs.")

    # 오늘 크롤링한 URL을 기준으로 상태를 설정
    log_data_deleted = []  # deleted 상태를 따로 저장
    log_data_other = []    # 나머지 (exist, update) 상태를 따로 저장

    # 오늘 크롤링한 링크와 이전 로그의 비교
    all_urls_set = set(all_links)
    previous_urls_set = set(previous_urls.keys())

    # 1. 존재하는 링크 처리 (오늘만 존재)
    existing_urls = all_urls_set & previous_urls_set  # 오늘과 어제 모두 존재하는 링크
    for url in existing_urls:
        notice_status = "exist"
        work_status = previous_urls[url]['work_status']  # 이전의 상태 그대로
        done_time = previous_urls[url]['done_time']  # 이전의 done_time 그대로
        job = previous_urls[url]['job']  # 이전 로그에서 job 값 가져오기
        log_data_other.append(f"{job},{url},{notice_status},{work_status},{done_time}")

    log_process(f"Processed {len(existing_urls)} existing URLs.")

    # 2. 새로 추가된 링크 처리 (오늘만 있는 링크)
    new_urls = all_urls_set - previous_urls_set  # 오늘만 존재하는 링크
    for url in new_urls:
        notice_status = "update"
        work_status = "null"
        done_time = "null"
        job = job_for_links[url]  # 새 URL에는 해당 job_key 값을 사용
        log_data_other.append(f"{job},{url},{notice_status},{work_status},{done_time}")

    log_process(f"Processed {len(new_urls)} new URLs.")

    # 3. 삭제된 링크 처리 (어제 있었으나 오늘은 없는 링크)
    deleted_urls = previous_urls_set - all_urls_set  # 어제는 있었지만 오늘은 없는 것
    for url in deleted_urls:
        notice_status = "deleted"
        work_status = "done"
        done_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')  # 삭제된 시간을 현재 시간으로 설정
        job = previous_urls[url]['job']  # 이전 로그에서 job 값 가져오기
        log_data_deleted.append(f"{job},{url},{notice_status},{work_status},{done_time}")

    log_process(f"Processed {len(deleted_urls)} deleted URLs.")

    # 로그 파일 저장
    with open(today_log_file_name, 'w', encoding='utf-8') as file:
        # 헤더 작성
        file.write("job,url,notice_status,work_status,done_time\n")
        # deleted 항목을 먼저 기록
        for line in log_data_deleted:
            file.write(line + "\n")
        # 나머지 (exist, update) 항목을 그 뒤에 기록
        for line in log_data_other:
            file.write(line + "\n")

    log_process(f"Log file '{today_log_file_name}' has been written successfully.")

except Exception as e:
    # 오류 발생 시 오류 메시지 기록
    error_message = str(e)
    log_error(error_message)
    log_process(f"An error occurred: {error_message}")  
    
    
 # incruit url 크롤링 완성    


Found the most recent log file: 20241221.log
Reading existing log file: 20241221.log
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=0 (page 0)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=30 (page 1)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=60 (page 2)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=90 (page 3)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=120 (page 4)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=150 (page 5)
Fetching URL: https://search.incruit.com/list/search.asp?col=job&kw=%B5%A5%C0%CC%C5%CD+%BF%A3%C1%F6%B4%CF%BE%EE&startno=18

In [1]:
import re
import os
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


if not os.path.exists('./wanted'):
    os.makedirs('./wanted')  # 디렉토리 생성  

def log_error(error_message):
    """오류를 날짜별로 makelog_err_YYYYMMDD.log 파일에 기록"""
    today = datetime.today().strftime('%Y%m%d')  # 날짜 포맷
    log_filename = f'./wanted/makelog_err_{today}.log'  # 파일명에 날짜 추가
    with open(log_filename, 'a', encoding='utf-8') as err_file:
        timestamp = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        err_file.write(f"{timestamp},{error_message}\n")

def log_process(process_message):
    """프로세스 로그를 날짜별로 makelog_process_YYYYMMDD.log 파일에 기록"""
    today = datetime.today().strftime('%Y%m%d')  # 날짜 포맷
    log_filename = f'./wanted/makelog_process_{today}.log'  # 파일명에 날짜 추가
    with open(log_filename, 'a', encoding='utf-8') as process_file:
        timestamp = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        process_file.write(f"{timestamp},{process_message}\n")

job_url_list = {
    "DE": [
        "https://www.wanted.co.kr/search?query=데이터+엔지니어&tab=position",
    ],
    "FE": [
        "https://www.wanted.co.kr/search?query=프론트엔드&tab=position"
    ],
    "BE": [
        "https://www.wanted.co.kr/search?query=백엔드&tab=position"
    ],
    "DA": [
        "https://www.wanted.co.kr/search?query=데이터+분석가&tab=position",
    ],
    "MLE": [
        "https://www.wanted.co.kr/search?query=머신러닝+엔지니어&tab=position",
    ]
}

try:
    # 셀레니움 웹 드라이버 설정
    options = Options()
    options.headless = False  # 드라이버를 헤드리스 모드로 실행할 수 있음 (주석 처리하거나 True로 설정하여 브라우저를 표시하지 않게 할 수 있음)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    log_process("Chrome WebDriver started.")

    # 오늘 날짜로 로그 파일 이름 설정
    today = datetime.today().strftime('%Y%m%d')
    today_log_file_name = f"./wanted/{today}.log"

    # 로그 파일을 찾을 디렉토리 설정
    log_directory = './wanted'  # 원하는 디렉토리로 변경
    log_files = [f for f in os.listdir(log_directory) if re.match(r'^\d{8}\.log$', f)]

    # 가장 최근에 생성된 로그 파일 찾기
    if log_files:
        log_files.sort(key=lambda x: os.path.getmtime(os.path.join(log_directory, x)), reverse=True)
        recent_log_file_name = log_files[0]  # 가장 최근의 로그 파일을 선택
        log_process(f"Found the most recent log file: {recent_log_file_name}")
    else:
        log_process("No log files found in the directory. All URLs will be marked as 'update'.")
        recent_log_file_name = None  # recent_log_file_name을 None으로 설정

    # 이전 로그 파일이 존재하는지 확인하고 읽기
    previous_urls = {}  # 이전 로그에 있는 URL 및 해당 job
    if recent_log_file_name and os.path.exists(os.path.join(log_directory, recent_log_file_name)):
        with open(os.path.join(log_directory, recent_log_file_name), 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if lines:  # 파일에 내용이 있을 때만 처리
                for line in lines[1:]:  # 첫 번째 줄은 header
                    columns = line.strip().split(',')
                    if len(columns) >= 5:  # 만약 각 열이 다 있다면
                        url = columns[1]
                        job = columns[0]  # 해당 URL의 job
                        notice_status = columns[2]
                        work_status = columns[3]
                        done_time = columns[4]
                        if notice_status != "deleted":  # "deleted" 상태인 URL은 제외
                            previous_urls[url] = {'job': job, 'notice_status': notice_status, 'work_status': work_status, 'done_time': done_time}  # URL에 해당하는 정보 저장

    # 오늘 크롤링한 URL을 수집
    all_links = []  # 오늘 수집한 모든 링크
    job_for_links = {}  # 각 링크에 해당하는 job을 기록하기 위한 dictionary

    # 각 job (키값)에 대한 URL 처리
    for job_key, urls in job_url_list.items():
        for url in urls:
            log_process(f"Processing job: {job_key}, URL: {url}")
            # 페이지 열기
            driver.get(url)

            # 페이지 로딩 대기: 페이지가 완전히 로드될 때까지 기다기
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "a"))
            )
            log_process(f"Page loaded: {url}")

            # 정규 표현식 패턴 (wd/ 뒤에 숫자가 있는 URL을 찾는 패턴)
            pattern = re.compile(r'wd/\d+$')

            # 스크롤 내리기 및 링크 추출 반복
            previous_height = driver.execute_script("return document.body.scrollHeight")  # 현재 페이지의 높이를 가져옴

            while True:
                # 페이지에서 모든 <a> 태그를 찾음
                links = driver.find_elements(By.TAG_NAME, "a")

                # 이미 가져온 링크들을 확인하고 중복되지 않게 추가
                for link in links:
                    href = link.get_attribute("href")
                    if href and pattern.search(href) and href not in all_links:
                        all_links.append(href)
                        job_for_links[href] = job_key  # 링크에 해당하는 job 기록

                # 스크롤을 페이지 끝까지 내리기
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                log_process(f"Scrolling page: {url}")

                # 잠시 대기하여 새로운 요소들이 로드될 시간을 줌
                time.sleep(2)  # 2초간 대기, 이 시간은 페이지 로딩 속도에 맞게 조절

                # 새로운 페이지 높이가 이전과 같다면 스크롤을 더 이상 내릴 필요가 없으므로 종료
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == previous_height:
                    break  # 더 이상 새로운 요소가 로드되지 않으면 반복 종료

                previous_height = new_height  # 이전 높이를 업데이트

    log_process(f"Total {len(all_links)} links collected today.")

    # 오늘 크롤링한 URL과 최근 로그 파일을 비교하여 상태 설정
    log_data_deleted = []  # deleted 상태를 따로 저장
    log_data_other = []    # 나머지 (exist, update) 상태를 따로 저장

    # 1. 오늘 크롤링한 URL과 이전 로그 파일의 비교
    for url in all_links:
        if url in previous_urls:
            # 이전 로그 파일과 오늘 모두 존재하는 URL이면 "exist"로 처리
            if previous_urls[url]['notice_status'] == "deleted":
                # 이미 'deleted' 상태로 존재하는 공고는 다시 "deleted"로 처리하지 않음
                continue
            notice_status = "exist"
            work_status = previous_urls[url]['work_status']  # 이전의 상태 그대로
            done_time = previous_urls[url]['done_time']  # 이전의 done_time 그대로
            job = previous_urls[url]['job']  # 이전 로그에서 job 값 가져오기
            log_data_other.append(f"{job},{url},{notice_status},{work_status},{done_time}")
        else:
            # 오늘만 존재하는 URL은 "update"로 설정
            notice_status = "update"
            work_status = "null"
            done_time = "null"
            job = job_for_links[url]  # 새 URL에는 해당 job_key 값을 사용
            log_data_other.append(f"{job},{url},{notice_status},{work_status},{done_time}")  # 새 URL에는 해당 job_key 값을 사용

    # 2. 이전 로그 파일에 있지만 오늘 로그 파일에 없는 URL 처리
    for url in previous_urls:
        if url not in all_links:
            # 이전에는 존재했지만 오늘은 없는 URL은 "deleted"로 설정
            if previous_urls[url]['notice_status'] == "deleted":
                # 이미 'deleted' 상태로 기록된 공고는 다시 'deleted'로 갱신하지 않음
                continue
            notice_status = "deleted"
            work_status = "done"
            done_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')  # 삭제된 시간을 현재 시간으로 설정
            job = previous_urls[url]['job']  # 이전 로그에서 job 값 가져오기
            log_data_deleted.append(f"{job},{url},{notice_status},{work_status},{done_time}")  # 삭제된 URL은 따로 추가

    # 로그 파일 저장
    with open(today_log_file_name, 'w', encoding='utf-8') as file:
        # 헤더 작성
        file.write("job,url,notice_status,work_status,done_time\n")
        # deleted 항목을 먼저 기록
        for line in log_data_deleted:
            file.write(line + "\n")
        # 나머지 (exist, update) 항목을 그 뒤에 기록
        for line in log_data_other:
            file.write(line + "\n")

    log_process(f"Log file saved: {today_log_file_name}")

    # 브라우저 종료
    driver.quit()
    log_process("Chrome WebDriver closed.")

except Exception as e:
    # 오류 발생 시 오류 메시지 기록
    error_message = str(e)
    log_error(error_message)
    log_process(f"Error occurred: {error_message}")
    # 프로그램 종료 전 브라우저 종료
    if 'driver' in locals():
        driver.quit()
        log_process("Chrome WebDriver closed after error.")
        
#wanted 완료


In [2]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 검색 키워드와 직무 약어 설정
search_keywords = {
    "데이터 엔지니어": "DE",
    "프론트엔드": "FE",
    "백엔드": "BE",
    "데이터분석가": "DA",
    "머신러닝 엔지니어": "MLE"
}

# 오늘 날짜로 txt 파일 이름 설정
today_date = datetime.now().strftime("%Y%m%d")

# ./jobkorea 폴더 생성 (존재하지 않으면 생성)
if not os.path.exists('./jobkorea'):
    os.makedirs('./jobkorea')

# 기존 링크 불러오기 (로컬 파일에서)
def load_existing_links(file_path):
    existing_links = set()

    # 로컬 파일에서 링크 읽기
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_links.update(line.strip() for line in file.readlines())
        print(f"기존 링크를 로드했습니다. 총 {len(existing_links)}개의 링크.")
    else:
        print(f"{file_path} 파일이 존재하지 않아 새로 생성합니다.")

    return existing_links

# 로컬에 텍스트 데이터 저장 함수
def save_to_local(data, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(data)
        print(f"파일이 로컬에 성공적으로 저장되었습니다. 경로: {file_path}")
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {e}")

# 특정 키워드에 대해 링크 크롤링
def scrape_links_for_keyword(keyword, keyword_code, existing_links):
    base_url = f"https://www.jobkorea.co.kr/Search/?stext={keyword}&tabType=recruit&Page_No="
    new_links = []  # 새로 크롤링한 URL 저장
    page_number = 1  # 첫 페이지 번호

    while True:
        # 페이지 열기
        url = f"{base_url}{page_number}"
        driver.get(url)
        time.sleep(3)  # 페이지 로딩 대기

        # 검색 결과가 없으면 종료
        try:
            no_results_message = driver.find_element(By.CLASS_NAME, "list-empty-result")
            if "검색결과가 없습니다" in no_results_message.text:
                print(f"{page_number}에 수집할 링크가 없으므로 링크 수집을 멈춥니다.")
                break
        except Exception:
            print(f"페이지 {page_number}: 검색 결과가 있습니다. 계속 진행합니다...")

        # article class="list" 내부의 링크 가져오기
        page_links = []  # 해당 페이지에서 새로 수집한 링크 저장
        try:
            article_list = driver.find_element(By.CLASS_NAME, "list")  # article class="list" 선택
            links = article_list.find_elements(By.TAG_NAME, "a")  # 해당 article 내부의 <a> 태그 찾기

            for link in links:
                href = link.get_attribute("href")
                if href and href.startswith("https://www.jobkorea.co.kr/Recruit/GI_Read"):
                    # 'PageGbn=HH'가 포함된 링크는 제외
                    if "?PageGbn=HH" not in href:
                        # URL에서 "?"로 나눠서 앞부분만 가져오기
                        clean_href = href.split('?')[0]
                        if clean_href not in existing_links:
                            page_links.append(clean_href)  # 직무 코드 없이 URL만 추가
                            existing_links.add(clean_href)  # 중복 방지
        except Exception as e:
            print(f"페이지 {page_number}에서 링크를 가져오는 데 오류가 발생했습니다: {e}")

        # 해당 페이지의 데이터를 저장
        if page_links:
            new_links.extend(page_links)

        # 페이지 번호 증가
        page_number += 1

    # 새 링크를 직무 코드별로 로컬 파일에 저장
    if new_links:
        file_path = f"./jobkorea/{keyword_code}_{today_date}.txt"
        output_data = "\n".join(new_links)
        save_to_local(output_data, file_path)
        print(f"{keyword} ({keyword_code})에서 수집한 링크를 로컬에 저장했습니다: {file_path}")

    return new_links

# 모든 키워드에 대해 크롤링
def scrape_links():
    all_new_links = []

    for keyword, keyword_code in search_keywords.items():
        # 기존 링크 로드 (각 직무 코드별로 별도 파일)
        file_path = f"./jobkorea/{keyword_code}_{today_date}.txt"
        existing_links = load_existing_links(file_path)
        
        print(f"검색 키워드: {keyword} ({keyword_code})")
        new_links = scrape_links_for_keyword(keyword, keyword_code, existing_links)
        all_new_links.extend(new_links)
        print(f"{keyword}에서 {len(new_links)}개의 새로운 링크를 발견하였습니다.")

    # 총 결과 출력
    print(f"총 {len(all_new_links)}개의 새로운 링크를 발견하였습니다.")

# 크롤링 실행
scrape_links()

# 브라우저 종료
driver.quit()

#잡코리아완료


./jobkorea/DE_20241222.txt 파일이 존재하지 않아 새로 생성합니다.
검색 키워드: 데이터 엔지니어 (DE)
81에 수집할 링크가 없으므로 링크 수집을 멈춥니다.
파일이 로컬에 성공적으로 저장되었습니다. 경로: ./jobkorea/DE_20241222.txt
데이터 엔지니어 (DE)에서 수집한 링크를 로컬에 저장했습니다: ./jobkorea/DE_20241222.txt
데이터 엔지니어에서 1165개의 새로운 링크를 발견하였습니다.
./jobkorea/FE_20241222.txt 파일이 존재하지 않아 새로 생성합니다.
검색 키워드: 프론트엔드 (FE)
82에 수집할 링크가 없으므로 링크 수집을 멈춥니다.
파일이 로컬에 성공적으로 저장되었습니다. 경로: ./jobkorea/FE_20241222.txt
프론트엔드 (FE)에서 수집한 링크를 로컬에 저장했습니다: ./jobkorea/FE_20241222.txt
프론트엔드에서 1326개의 새로운 링크를 발견하였습니다.
./jobkorea/BE_20241222.txt 파일이 존재하지 않아 새로 생성합니다.
검색 키워드: 백엔드 (BE)
116에 수집할 링크가 없으므로 링크 수집을 멈춥니다.
파일이 로컬에 성공적으로 저장되었습니다. 경로: ./jobkorea/BE_20241222.txt
백엔드 (BE)에서 수집한 링크를 로컬에 저장했습니다: ./jobkorea/BE_20241222.txt
백엔드에서 1817개의 새로운 링크를 발견하였습니다.
./jobkorea/DA_20241222.txt 파일이 존재하지 않아 새로 생성합니다.
검색 키워드: 데이터분석가 (DA)
7에 수집할 링크가 없으므로 링크 수집을 멈춥니다.
파일이 로컬에 성공적으로 저장되었습니다. 경로: ./jobkorea/DA_20241222.txt
데이터분석가 (DA)에서 수집한 링크를 로컬에 저장했습니다: ./jobkorea/DA_20241222.txt
데이터분석가에서 64개의 새로운 링크를 발견하였습니다.
./jobkorea/MLE_20241

In [4]:
import re
import os
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 검색 키워드 설정
search_keyword = "데이터 엔지니어"

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Jumpit 페이지 열기
driver.get("https://jumpit.saramin.co.kr/")

# 페이지 로딩 대기
time.sleep(5)

# 팝업 닫기
try:
    close_popup = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sc-c82c57bb-4.cLpfjF"))
    )
    close_popup.click()
    print("팝업 닫음.")
except Exception as e:
    print(f"팝업 닫기 실패: {e}")

# 검색창 찾기 및 검색
try:
    search_input = driver.find_element(By.CSS_SELECTOR, "input[placeholder='검색어를 입력해주세요']")
    search_input.send_keys(search_keyword)  # 검색어 입력
    search_input.send_keys(Keys.RETURN)  # Enter 키 입력
    print(f"'{search_keyword}' 검색어 입력 완료.")
except Exception as e:
    print(f"검색창을 찾을 수 없습니다: {e}")
    driver.quit()
    exit()

# 검색 결과 로딩 대기
time.sleep(5)

# 링크와 D-day 값을 저장할 딕셔너리
link_data = {}

# 정규 표현식 패턴 (position/ 뒤에 숫자가 있는 URL을 찾는 패턴)
pattern = re.compile(r'position/\d+$')

# 스크롤 내리기 및 링크 추출 반복
previous_height = driver.execute_script("return document.body.scrollHeight")

while True:
    links = driver.find_elements(By.TAG_NAME, "a")
    for link in links:
        href = link.get_attribute("href")
        if href and pattern.search(href) and href not in link_data:
            # D-day 정보 추출
            try:
                d_day_elements = link.find_elements(By.XPATH, ".//div[contains(@class, 'sc-d609d44f-3')]/span")
                if d_day_elements:
                    d_day_text = d_day_elements[0].text.strip()
                    if "상시" in d_day_text:
                        d_day = None  # 상시는 null로 저장
                    elif d_day_text.startswith("D-"):
                        d_day = int(d_day_text[2:])  # D-숫자에서 숫자만 추출
                    else:
                        d_day = None
                else:
                    d_day = None
            except Exception as e:
                print(f"D-day 추출 실패: {e}")
                d_day = None

            link_data[href] = d_day
            print(f"링크: {href}, D-day: {d_day}")

    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 확인
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == previous_height:
        break
    previous_height = new_height

# 디렉터리 생성 (로컬 저장용 디렉터리)
output_folder = "./jumpit"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 오늘 날짜 기반 파일 이름 생성
today = datetime.now().strftime("%Y%m%d")
output_file_path = os.path.join(output_folder, f"{today}.log")
log_file_name = os.path.join(output_folder, f"{today}.log")

yesterday = (datetime.today() - timedelta(days=1)).strftime('%Y%m%d')
yesterday_log_file = os.path.join(output_folder, f"{yesterday}.log")

# 어제 로그 파일이 있으면 읽기
previous_urls = {}
if os.path.exists(yesterday_log_file):
    with open(yesterday_log_file, 'r', encoding='utf-8') as file:
        for line in file.readlines()[1:]:  # 첫 번째 줄은 header
            columns = line.strip().split(',')
            url = columns[0]
            notice_status = columns[1]
            work_status = columns[2]
            done_time = columns[3]
            previous_urls[url] = {
                'notice_status': notice_status,
                'work_status': work_status,
                'done_time': done_time
            }

# 오늘 로그 파일에 기록할 내용 생성
log_data = []

# 오늘 크롤링한 URL과 어제 로그 파일을 비교하여 상태 설정
for url, d_day in link_data.items():
    if os.path.exists(yesterday_log_file):
        if url in previous_urls:
            if previous_urls[url]['work_status'] != "done":
                notice_status = previous_urls[url]['notice_status']
                work_status = previous_urls[url]['work_status']
                done_time = previous_urls[url]['done_time']
            else:
                notice_status = "exist"
                work_status = "done"
                done_time = previous_urls[url]['done_time']
        else:
            notice_status = "deleted"
            work_status = "done"
            done_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    else:
        notice_status = "update"
        work_status = "null"
        done_time = "null"

    # D-day 값을 로그 데이터의 마지막 열로 추가
    log_data.append(f"{url},{notice_status},{work_status},{done_time},{d_day if d_day is not None else 'null'}")

# 오늘 로그 파일 생성 (기존 로그 파일 덮어쓰기)
with open(log_file_name, 'w', encoding='utf-8') as file:
    # 헤더 작성
    file.write("url,notice_status,work_status,done_time,d_day\n")
    for line in log_data:
        file.write(line + "\n")

print(f"총 {len(log_data)}개의 링크와 D-day 정보가 저장되었습니다. 파일: {output_file_path}")

# Selenium 종료
driver.quit()


# 점핏 완료


팝업 닫음.
'데이터 엔지니어' 검색어 입력 완료.
링크: https://jumpit.saramin.co.kr/position/40484, D-day: 28
링크: https://jumpit.saramin.co.kr/position/40483, D-day: 28
링크: https://jumpit.saramin.co.kr/position/40482, D-day: 28
링크: https://jumpit.saramin.co.kr/position/40532, D-day: 29
링크: https://jumpit.saramin.co.kr/position/38294, D-day: 2
링크: https://jumpit.saramin.co.kr/position/40534, D-day: 29
링크: https://jumpit.saramin.co.kr/position/39493, D-day: 13
링크: https://jumpit.saramin.co.kr/position/38306, D-day: 2
링크: https://jumpit.saramin.co.kr/position/39682, D-day: 22
링크: https://jumpit.saramin.co.kr/position/39956, D-day: 21
D-day 추출 실패: invalid literal for int() with base 10: 'day'
링크: https://jumpit.saramin.co.kr/position/39038, D-day: None
링크: https://jumpit.saramin.co.kr/position/38103, D-day: 5
링크: https://jumpit.saramin.co.kr/position/40091, D-day: 23
링크: https://jumpit.saramin.co.kr/position/40092, D-day: 23
링크: https://jumpit.saramin.co.kr/position/40152, D-day: 24
링크: https://jumpit.saramin.c

In [5]:
import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from zoneinfo import ZoneInfo
import re

# 변수 설정 : 검색 키워드
job_titles = {
    "DE": "데이터 엔지니어",
    "DA": "데이터 분석가",
    "FE": "프론트엔드 엔지니어",
    "BE": "백엔드 엔지니어",
    "MLE": "머신러닝 엔지니어"
}

# 변수 설정 : 로컬 저장 경로
LOCAL_PATH = './rocketpunch/'

kst = ZoneInfo("Asia/Seoul")

# Chrome driver 옵션 설정
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Headless 모드
chrome_options.add_argument("--disable-gpu")  # GPU 비활성화
chrome_options.add_argument("--no-sandbox")  # 리소스 제약 없는 환경에서 실행

def get_total_pages(base_url):
    """ Rocketpunch의 유효한 페이지 수를 반환하는 함수."""
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    try:
        page = 1
        while True:
            page_url = f"{base_url}&page={page}"
            driver.get(page_url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#search-results > div.ui.job.items.segment.company-list > div.company.item'))
            )
            job_elements = driver.find_elements(By.CSS_SELECTOR, '#search-results > div.ui.job.items.segment.company-list > div.company.item')
            if not job_elements:
                print(f"Page {page} has no job postings. Total pages: {page - 1}")
                return page - 1
            page += 1
    except TimeoutException:
        print(f"Timeout on page {page}")
        return page - 1
    except WebDriverException as e:
        print(f"WebDriverException: {e}")
        return page - 1
    finally:
        driver.quit()

def get_links_from_page(page_url):
    """ 
    한 페이지에서 채용공고 링크를 크롤링하는 함수
    :param page_url: 특정 페이지 URL
    :return: 페이지 내 공고 링크 리스트
    """
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    try:
        driver.get(page_url)

        # 모든 공고가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#search-results > div.ui.job.items.segment.company-list > div.company.item'))
        )

        links = driver.find_elements(By.TAG_NAME, "a")
        pattern = re.compile(r"^https://www\.rocketpunch\.com/jobs/\d+/[\w\-\%]+")
        page_links = [link.get_attribute("href") for link in links if link.get_attribute("href") and pattern.search(link.get_attribute("href"))]
        return page_links
    
    except TimeoutException:
        print(f"Timeout while loading page: {page_url}")
        return []
    except WebDriverException as e:
        print(f"WebDriver error while accessing page: {e}")
        return []
    finally:
        driver.quit()

def get_all_links(keyword):
    """ 모든 페이지에서 채용공고 링크를 수집하는 함수, 나중에 직무(DA, DE, FE, BE...) 에 따라 공고 링크 수집"""
    all_posts_links = []  # 데이터 엔지니어 공고 링크 리스트

    keyword = keyword.replace(" ", "+")
    base_url = f"https://www.rocketpunch.com/jobs?keywords={keyword}"  # "데이터" || "엔지니어"
    
    try:
        total_pages = get_total_pages(base_url)
        if total_pages == 0:
            print(f"키워드 {keyword}에 해당하는 채용공고가 없습니다!")
            return []

        for page in range(1, total_pages + 1):
            page_url = f"{base_url}&page={page}"
            page_links = get_links_from_page(page_url)
            all_posts_links.extend(page_links)

        return all_posts_links

    except Exception as e:
        print(f"Unexpected error occurred: {e}")
        return []

def save_link_to_local(local_path, today_date, today_links):
    """ 로컬에 파일을 저장하는 함수
    :param local_path: 로컬 디렉터리 경로
    :param today_date: 오늘 날짜 (파일 이름)
    :param today_links: 저장할 링크 리스트
    :return: 파일 저장 성공 여부
    """
    try:
        if not os.path.exists(local_path):
            os.makedirs(local_path)  # 경로가 없으면 생성
        
        file_name = f"{today_date}.txt"
        file_path = os.path.join(local_path, file_name)

        with open(file_path, 'w', encoding='utf-8') as file:
            file.write("\n".join(today_links))
        
        print(f"✅ 링크 파일 {file_name}이 로컬에 성공적으로 저장되었습니다")
        return True

    except Exception as e:
        print(f"⛔ [ERROR] 로컬로 파일을 저장하는데 에러 발생 {e}")
        return False

# 코드가 바로 실행되도록 변경된 부분
for job_abb, job_title in job_titles.items():
    # 로컬 링크 저장 경로 설정
    local_path = os.path.join(LOCAL_PATH, job_abb)
    today_links = get_all_links(job_title)

    today_date = datetime.now(tz=kst).strftime('%Y%m%d')
    # 오늘 수집한 링크를 로컬에 저장
    save_link_to_local(local_path, today_date, today_links)
# 로켓펀치 완료 

Timeout on page 8
✅ 링크 파일 20241222.txt이 로컬에 성공적으로 저장되었습니다
Timeout on page 8
✅ 링크 파일 20241222.txt이 로컬에 성공적으로 저장되었습니다
Timeout on page 5
✅ 링크 파일 20241222.txt이 로컬에 성공적으로 저장되었습니다
Timeout on page 6
✅ 링크 파일 20241222.txt이 로컬에 성공적으로 저장되었습니다
Timeout on page 4
✅ 링크 파일 20241222.txt이 로컬에 성공적으로 저장되었습니다


In [10]:
import re
import os
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import unquote

if not os.path.exists('./saramin'):
    os.makedirs('./saramin')  # 디렉토리 생성

def log_error(error_message):
    """오류를 날짜별로 makelog_err_YYYYMMDD.log 파일에 기록"""
    today = datetime.today().strftime('%Y%m%d')  # 날짜 포맷
    log_filename = f'./saramin/makelog_err_{today}.log'  # 파일명에 날짜 추가
    with open(log_filename, 'a', encoding='utf-8') as err_file:
        timestamp = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        err_file.write(f"{timestamp},{error_message}\n")

def log_process(process_message):
    """프로세스 로그를 날짜별로 makelog_process_YYYYMMDD.log 파일에 기록"""
    today = datetime.today().strftime('%Y%m%d')  # 날짜 포맷
    log_filename = f'./saramin/makelog_process_{today}.log'  # 파일명에 날짜 추가
    with open(log_filename, 'a', encoding='utf-8') as process_file:
        timestamp = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        process_file.write(f"{timestamp},{process_message}\n")

def convert_html_entities_to_url(html_link):
    """HTML 엔티티를 URL로 변환하고, 상대경로를 절대경로로 변환"""
    # HTML 엔티티 &amp;를 &로 변환
    html_link = html_link.replace('&amp;', '&')

    # 상대 경로에 도메인 추가
    base_url = "https://www.saramin.co.kr"
    if html_link.startswith('/'):
        html_link = base_url + html_link

    # URL 인코딩된 값 복원 (예: %2F -> /)
    html_link = unquote(html_link)

    return html_link

def has_uuid(url):
    """URL에 uuid가 포함되어 있는지 확인"""
    return bool(re.search(r'[0-9a-fA-F-]{36}', url))  # UUID 형식을 정규표현식으로 검사

# 글로벌 변수 선언
job_url_list = {
    "DE": [
        "https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword=데이터%20엔지니어&recruitPage={}&recruitSort=relation&recruitPageCount=40&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting=&mainSearch=n",
    ],
    "FE": [
        "https://www.saramin.co.kr/zf_user/search/recruit?searchType=search&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&keydownAccess=&searchword=프론트엔드&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage={}&recruitSort=relation&recruitPageCount=40&inner_com_type=&show_applied=&quick_apply=&except_read=&ai_head_hunting=",
    ],
    "BE": [
        "https://www.saramin.co.kr/zf_user/search/recruit?searchType=search&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&keydownAccess=&searchword=백엔드&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage={}&recruitSort=relation&recruitPageCount=40&inner_com_type=&show_applied=&quick_apply=&except_read=&ai_head_hunting=",
    ],
    "MLE": [
        "https://www.saramin.co.kr/zf_user/search/recruit?searchType=search&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&keydownAccess=&searchword=머신러닝%20엔지니어&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage={}&recruitSort=relation&recruitPageCount=40&inner_com_type=&show_applied=&quick_apply=&except_read=&ai_head_hunting=",
    ],
    "DA": [
        "https://www.saramin.co.kr/zf_user/search/recruit?searchType=search&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&keydownAccess=&searchword=데이터%20분석가&panel_type=&search_optional_item=y&search_done=y&panel_count=y&preview=y&recruitPage={}&recruitSort=relation&recruitPageCount=40&inner_com_type=&show_applied=&quick_apply=&except_read=&ai_head_hunting=",
    ]
}

try:
    # 셀레니움 웹 드라이버 설정
    options = Options()
    options.headless = False  # 드라이버를 헤드리스 모드로 실행할 수 있음 (True로 설정하여 브라우저를 표시하지 않게 할 수 있음)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    log_process("Chrome WebDriver started.")

    # 오늘 날짜로 로그 파일 이름 설정
    today = datetime.today().strftime('%Y%m%d')
    today_log_file_name = f"./saramin/{today}.log"

    # 로그 파일을 찾을 디렉토리 설정
    log_directory = './saramin'  # 원하는 디렉토리로 변경
    log_files = [f for f in os.listdir(log_directory) if re.match(r'^\d{8}\.log$', f)]

    # 가장 최근에 생성된 로그 파일 찾기
    if log_files:
        log_files.sort(key=lambda x: os.path.getmtime(os.path.join(log_directory, x)), reverse=True)
        recent_log_file_name = log_files[0]  # 가장 최근의 로그 파일을 선택
        log_process(f"Found the most recent log file: {recent_log_file_name}")
    else:
        log_process("No log files found in the directory. All URLs will be marked as 'update'.")
        recent_log_file_name = None  # recent_log_file_name을 None으로 설정

    # 이전 로그 파일이 존재하는지 확인하고 읽기
    previous_urls = {}  # 이전 로그에 있는 URL 및 해당 job
    if recent_log_file_name and os.path.exists(os.path.join(log_directory, recent_log_file_name)):
        with open(os.path.join(log_directory, recent_log_file_name), 'r', encoding='utf-8') as file:
            lines = file.readlines()
            if lines:  # 파일에 내용이 있을 때만 처리
                for line in lines[1:]:  # 첫 번째 줄은 header
                    columns = line.strip().split(',')
                    if len(columns) >= 5:  # 만약 각 열이 다 있다면
                        url = columns[1]
                        job = columns[0]  # 해당 URL의 job
                        notice_status = columns[2]
                        work_status = columns[3]
                        done_time = columns[4]
                        if notice_status != "deleted":  # "deleted" 상태인 URL은 제외
                            previous_urls[url] = {'job': job, 'notice_status': notice_status, 'work_status': work_status, 'done_time': done_time}  # URL에 해당하는 정보 저장

    # 오늘 크롤링한 URL을 수집
    all_links = []  # 오늘 수집한 모든 링크
    job_for_links = {}  # 각 링크에 해당하는 job을 기록하기 위한 dictionary

    # 각 job (키값)에 대한 URL 처리
    for job_key, urls in job_url_list.items():
        for url in urls:
            log_process(f"Processing job: {job_key}, URL: {url}")
            for page_number in range(1, 41):  # 1부터 40까지 페이지 크롤링
                page_url = url.format(page_number)
                driver.get(page_url)

                # 페이지 로딩 대기
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "a")))

                # 링크 추출
                links = driver.find_elements(By.TAG_NAME, "a")
                for link in links:
                    href = link.get_attribute("href")
                    if href:
                        converted_url = convert_html_entities_to_url(href)

                        # UUID가 포함된 링크만 추가
                        if has_uuid(converted_url) and converted_url not in all_links:
                            all_links.append(converted_url)
                            job_for_links[converted_url] = job_key  # 링크와 job 정보를 매핑

    # 크롤링 완료 후, 링크 정보를 로그로 기록
    for link in all_links:
        job = job_for_links[link]
        log_process(f"{job}, {link}")

    # 오늘 크롤링한 URL과 이전 로그 파일을 비교하여 상태 설정
    log_data_deleted = []  # deleted 상태를 따로 저장
    log_data_other = []    # 나머지 (exist, update) 상태를 따로 저장

    # 1. 오늘 크롤링한 URL과 이전 로그 파일의 비교
    for url in all_links:
        if url in previous_urls:
            # 이전 로그 파일과 오늘 모두 존재하는 URL이면 "exist"로 처리
            if previous_urls[url]['notice_status'] == "deleted":
                continue
            notice_status = "exist"
            work_status = previous_urls[url]['work_status']
            done_time = previous_urls[url]['done_time']
            job = previous_urls[url]['job']
            log_data_other.append(f"{job},{url},{notice_status},{work_status},{done_time}")
        else:
            # 오늘만 존재하는 URL은 "update"로 설정
            notice_status = "update"
            work_status = "null"
            done_time = "null"
            job = job_for_links[url]
            log_data_other.append(f"{job},{url},{notice_status},{work_status},{done_time}")

    # 2. 이전 로그 파일에 있지만 오늘 로그 파일에 없는 URL 처리
    for url in previous_urls:
        if url not in all_links:
            notice_status = "deleted"
            work_status = "done"
            done_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            job = previous_urls[url]['job']
            log_data_deleted.append(f"{job},{url},{notice_status},{work_status},{done_time}")

    # 로그 파일 저장
    with open(today_log_file_name, 'w', encoding='utf-8') as file:
        file.write("job,url,notice_status,work_status,done_time\n")
        for line in log_data_deleted:
            file.write(line + "\n")
        for line in log_data_other:
            file.write(line + "\n")

    log_process(f"Log file saved: {today_log_file_name}")

except Exception as e:
    # 오류 발생 시 오류 메시지 기록
    error_message = str(e)
    log_error(error_message)
    log_process(f"Error occurred: {error_message}")
    if 'driver' in locals():
        driver.quit()
        log_process("Chrome WebDriver closed after error.")

finally:
    # 드라이버 종료 (예외 없이 처리되었을 경우에도)
    if 'driver' in locals():
        driver.quit()
        log_process("Chrome WebDriver closed.")   

# saramin done 

KeyboardInterrupt: 